In [ ]:
import boto3
import io
import pandas as pd
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

region = boto3.session.Session().region_name
role = get_execution_role()

sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.m5.large", instance_count=5
)

In [ ]:
import pandas as pd

input_data = "s3://sagemaker-kproject/data/KR7000210005.csv".format(region)
df = pd.read_csv(input_data)
df

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2021/12/27,210,DL,"63,000","63,900","64,400","62,600","1,320,220,692,000","54,725","13,077",4.82,"67,178",0.94,"1,300",2.06
1,2021/12/24,210,DL,"63,900","63,700","64,400","63,500","1,339,080,987,600","36,855","13,077",4.89,"67,178",0.95,"1,300",2.03
2,2021/12/23,210,DL,"63,700","63,300","63,700","63,000","1,334,889,810,800","25,141","13,077",4.87,"67,178",0.95,"1,300",2.04
3,2021/12/22,210,DL,"63,500","62,900","63,900","62,800","1,330,698,634,000","37,836","13,077",4.86,"67,178",0.95,"1,300",2.05
4,2021/12/21,210,DL,"62,800","63,800","63,900","62,200","1,316,029,515,200","46,287","13,077",4.80,"67,178",0.93,"1,300",2.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6747,1995/05/08,210,대림산업,"17,200","17,300","17,300","17,000","379,794,318,000","13,190",-,-,-,-,-,-
6748,1995/05/06,210,대림산업,"17,300","17,700","17,700","17,300","382,002,424,500","9,560",-,-,-,-,-,-
6749,1995/05/04,210,대림산업,"17,600","18,100","18,100","17,600","388,626,744,000","25,040",-,-,-,-,-,-
6750,1995/05/03,210,대림산업,"18,100","18,200","18,300","18,000","399,667,276,500","46,070",-,-,-,-,-,-


In [ ]:
df.shape

(6752, 15)

In [ ]:
%%writefile preprocessing.py

import argparse
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.exceptions import DataConversionWarning


warnings.filterwarnings(action='ignore', category=DataConversionWarning)


def print_shape(df):
    print('Data shape: {}'.format(df.shape))

        
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--train-test-split-ratio", type=float, default=0.3)
    args, _ = parser.parse_known_args()

    
    print("Received arguments {}".format(args))        
    input_data_path = os.path.join("/opt/ml/processing/input", "KR7000210005.csv")

    print("Reading input data from {}".format(input_data_path))
    df = pd.read_csv(input_data_path)
    df = pd.DataFrame(data=df)    
    
    
    print(df)
          
    print("data shape before preprocessing: {}".format(df.shape))

    # 날짜, 시가총액 열만 가져오기
    df = df[['TRD_DD','MKTCAP']]
    
    
    print("data shape after preprocessing: {}".format(df.shape))
    
    
    train_output_path = os.path.join('/opt/ml/processing/train', 'train.csv')

    test_output_path = os.path.join('/opt/ml/processing/test', 'test.csv')

    print('Saving train data {}'.format(train_output_path))

    pd.DataFrame(df).to_csv( train_output_path, index=False)

    print('Saving test data {}'.format(test_output_path))
    pd.DataFrame(df).to_csv(test_output_path, index=False)



Overwriting preprocessing.py


In [ ]:

sklearn_processor.run(
    code="preprocessing.py",
    inputs=[ProcessingInput(source='s3://sagemaker-kproject/data/KR7000210005.csv', 
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ]
    
)


Job Name:  sagemaker-scikit-learn-2022-01-03-16-10-23-559
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-kproject/data/KR7000210005.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-268367265700/sagemaker-scikit-learn-2022-01-03-16-10-23-559/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-2-268367265700/sagemaker-scikit-learn-2022-01-03-16-10-23-559/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test_data', 

In [ ]:
# 결과 확인

test_data = "s3://sagemaker-ap-northeast-2-268367265700/sagemaker-scikit-learn-2022-01-03-16-10-23-559/output/test_data/test.csv".format(region)
test = pd.read_csv(test_data)
test


,TRD_DD,MKTCAP
0,2021/12/27,"1,320,220,692,000"
1,2021/12/24,"1,339,080,987,600"
2,2021/12/23,"1,334,889,810,800"
3,2021/12/22,"1,330,698,634,000"
4,2021/12/21,"1,316,029,515,200"
...,...,...
6747,1995/05/08,"379,794,318,000"
6748,1995/05/06,"382,002,424,500"
6749,1995/05/04,"388,626,744,000"
6750,1995/05/03,"399,667,276,500"


In [ ]:
train_data = "s3://sagemaker-ap-northeast-2-268367265700/sagemaker-scikit-learn-2022-01-03-16-10-23-559/output/train_data/train.csv".format(region)
train = pd.read_csv(train_data)
train

,TRD_DD,MKTCAP
0,2021/12/27,"1,320,220,692,000"
1,2021/12/24,"1,339,080,987,600"
2,2021/12/23,"1,334,889,810,800"
3,2021/12/22,"1,330,698,634,000"
4,2021/12/21,"1,316,029,515,200"
...,...,...
6747,1995/05/08,"379,794,318,000"
6748,1995/05/06,"382,002,424,500"
6749,1995/05/04,"388,626,744,000"
6750,1995/05/03,"399,667,276,500"
